In [135]:
import base64
import datetime
import json
import os
import random
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

pd.options.display.max_colwidth = 1000

In [136]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix

from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy import sparse

# Load the Data

In [137]:
d = pd.concat([pd.read_csv('tmp/train_ids.csv', index_col='id'),
               pd.read_csv('tmp/test_ids.csv', index_col='id')])

In [138]:
e = pd.read_csv('tmp/features_bathrooms.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [139]:
e = pd.read_csv('tmp/features_bedrooms.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [140]:
e = pd.read_csv('tmp/features_building_id.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [141]:
e = pd.read_csv('tmp/features_created.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [142]:
e = pd.read_csv('tmp/features_description.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [143]:
e = pd.read_csv('tmp/features_display_address.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [144]:
e = pd.read_csv('tmp/features_features.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [145]:
e = pd.read_csv('tmp/features_latlon.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [146]:
e = pd.read_csv('tmp/features_manager_id.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [147]:
e = pd.read_csv('tmp/features_photos.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [148]:
e = pd.read_csv('tmp/features_price.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [149]:
e = pd.read_csv('tmp/features_street_address.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [150]:
d.interest_level.value_counts()

low       34284
medium    11229
high       3839
Name: interest_level, dtype: int64

In [153]:
# d = pd.concat([d, tf_idfs], axis=1)
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124011 entries, 0 to 124010
Columns: 103 entries, listing_id to daddr_saddr_same
dtypes: float64(49), int64(42), object(12)
memory usage: 98.4+ MB


In [154]:
tr = d[d.interest_level.notnull()].copy()
print len(tr)

49352


In [155]:
trl = tr.interest_level.copy()
trf = tr
del trf['interest_level']

In [156]:
te = d[d.interest_level.isnull()].copy()
print len(te)

74659


In [157]:
tef = te
del tef['interest_level']

In [158]:
categorical = ["manager_id", "building_id"]
for f in categorical:
        if trf[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(trf[f].values) + list(tef[f].values))
            trf[f] = lbl.transform(list(trf[f].values))
            tef[f] = lbl.transform(list(tef[f].values))
            


In [159]:
trf.columns

Index([u'listing_id', u'bathrooms', u'num_bathrooms_bucket', u'bedrooms',
       u'num_bedrooms_bucket', u'building_id', u'num_apts_in_building',
       u'num_apts_in_building_q10', u'created', u'created_utc',
       ...
       u'price_q10_all', u'price_q100_all', u'price_q10', u'price_q100',
       u'street_address', u'saddr_num_words', u'saddr_has_number',
       u'saddr_has_ordinal', u'saddr_num_upper_words', u'daddr_saddr_same'],
      dtype='object', length=102)

In [161]:

tfidf_features = CountVectorizer(stop_words='english', max_features=400)
tr_sparse_features = tfidf.fit_transform(trf["features"])
te_sparse_features = tfidf.transform(tef["features"])

tfidf_desc = CountVectorizer(stop_words='english', max_features=400)
tr_sparse_desc = tfidf.fit_transform(trf["desc_clean"].fillna(""))
te_sparse_desc = tfidf.transform(tef["desc_clean"].fillna(""))

In [162]:
for c in trf.columns:
    if trf.dtypes[c] not in ('int64', 'float64'):
        del trf[c]           
            
for c in tef.columns:
    if tef.dtypes[c] not in ('int64', 'float64'):
        del tef[c]

# Run Classification

In [163]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 4
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.8
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [166]:
train_X = sparse.hstack([trf,tr_sparse_features, tr_sparse_desc]).tocsr()
test_X = sparse.hstack([tef,te_sparse_features, te_sparse_desc]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(trl.apply(lambda x: target_num_map[x]))
train_y

array([1, 2, 1, ..., 1, 1, 0])

In [167]:
preds, model = runXGB(train_X, train_y, test_X, num_rounds=1000)

In [168]:
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = pd.read_csv('input/sample_submission.csv').listing_id.values
out_df.to_csv("output/xgb_starter_3_own_features_tfidf_desc_manager_building.csv", index=False)

In [169]:
def cvXGB(train_X, train_y, test_X, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 4
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.8
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    eval_hist = xgb.cv(plst, xgtrain, num_rounds, nfold=3)

    
    return eval_hist


In [170]:
eval_history = cvXGB(train_X, train_y, test_X, num_rounds=1000)
eval_history

,test-mlogloss-mean,test-mlogloss-std,train-mlogloss-mean,train-mlogloss-std
0,1.039409,0.000225,1.038958,0.000575
1,0.987257,0.000395,0.986396,0.000696
2,0.943749,0.000450,0.942460,0.001421
3,0.906008,0.000079,0.904247,0.001569
4,0.873241,0.000440,0.871052,0.001416
5,0.844407,0.000533,0.841728,0.001477
6,0.819448,0.000725,0.816391,0.001534
7,0.797570,0.000814,0.794120,0.001630
8,0.778447,0.001111,0.774565,0.001480
9,0.761174,0.001181,0.756849,0.001482
